In [44]:
using Sundials
using DataFrames
include("regimen.jl")

regimen (generic function with 2 methods)

In [27]:
function onecmptiv(t, a, adot, p)
	# p1 = KEL	
        adot[1] = -p[1]*a[1]
end

function sim_ind(f, regimen, sample_times)
	sample_times = copy(sample_times)
    conc = Array(Float64,0)
    time = Array(Float64,0)
	for (j,amt) in enumerate(regimen[2])
		dtime = (j == length(regimen[2]) ? sample_times[end] : regimen[1][j+1])
		time_slice = Float64[] # must define outside for loop or time_slice only defined inside will not be available ever in global scope
		for i in  1:length(sample_times) 
			if sample_times[i] .< dtime 
				continue
			else 
				time_slice = splice!(sample_times, 1:i-1) # as loop will go 1 past the last element we want only want to splice to i-1
				push!(time_slice, dtime)
				break
			end
		end
		if j == 1	
		y =Sundials.cvode(f, [amt], time_slice)
		else
		y =Sundials.cvode(f, [conc[end] + amt], time_slice)
		end
		append!(conc, y[1:end])
		append!(time, time_slice)
		
	end
    return (time, conc)
end


sim_ind (generic function with 1 method)

In [19]:
# analytical solution
function onecmptiv_analytical(ke, amt0, times)
    t0 = times[1]
    concs = Array(Float64,0)
    for i in 1:length(times)
        push!(concs, amt0*exp(-ke*(times[i]-t0)))
    end
    return(concs)
end

onecmptiv_analytical (generic function with 1 method)

In [18]:
onecmptiv_analytical(0.1, 7.4 + 10., [3, 4, 5, 6])

3

4-element Array{Float64,1}:
 17.4   
 15.7442
 14.2459
 12.8902

In [30]:

function sim_ind_analytical(cl, v, regimen, sample_times)
    ke = cl/v
	sample_times = copy(sample_times)
    conc = Array(Float64,0)
    time = Array(Float64,0)
	for (j,amt) in enumerate(regimen[2])
		dtime = (j == length(regimen[2]) ? sample_times[end] : regimen[1][j+1])
		time_slice = Float64[] # must define outside for loop or time_slice only defined inside will not be available ever in global scope
		for i in  1:length(sample_times) 
			if sample_times[i] .< dtime 
				continue
			else 
				time_slice = splice!(sample_times, 1:i-1) # as loop will go 1 past the last element we want only want to splice to i-1
				push!(time_slice, dtime)
				break
			end
		end
         if j == 1	
            #y =Sundials.cvode(f, [amt], time_slice)
            y = onecmptiv_analytical(ke, amt, time_slice)
        else
            y =onecmptiv_analytical(ke, conc[end] + amt, time_slice)
		end
		append!(conc, y[1:end])
		append!(time, time_slice)
		
	end
    return (time, conc)
end


sim_ind_analytical (generic function with 1 method)

In [31]:
reg = regimen(100., 5., interval = 12)
p = [0.1] # KEL
test_onecmpt(t, a, adot) = onecmptiv(t, a, adot, p)
sample_times = [0.:0.01:reg[1][end]+24.]



7201-element Array{Float64,1}:
  0.0 
  0.01
  0.02
  0.03
  0.04
  0.05
  0.06
  0.07
  0.08
  0.09
  0.1 
  0.11
  0.12
  ⋮   
 71.89
 71.9 
 71.91
 71.92
 71.93
 71.94
 71.95
 71.96
 71.97
 71.98
 71.99
 72.0 


 in depwarn at deprecated.jl:62
 in oldstyle_vcat_warning at abstractarray.jl:29
 in regimen at C:\Users\devin\Documents\Repos\julia-ode\regimen.jl:5
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at C:\Users\devin\.julia\v0.4\IJulia\src\execute_request.jl:157
 in eventloop at C:\Users\devin\.julia\v0.4\IJulia\src\IJulia.jl:123
 in anonymous at task.jl:365
while loading In[31], in expression starting on line 1


In [32]:
sim_ind(test_onecmpt, reg, sample_times)


([0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09  …  71.91,71.92,71.93,71.94,71.95,71.96,71.97,71.98,71.99,72.0],[100.0,99.9001,99.8003,99.7006,99.601,99.5015,99.4021,99.3028,99.2036,99.1045  …  13.0623,13.0493,13.0362,13.0232,13.0102,12.9972,12.9842,12.9712,12.9582,12.9453])

In [33]:
sim_ind_analytical(1, 10, reg, sample_times)

([0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09  …  71.91,71.92,71.93,71.94,71.95,71.96,71.97,71.98,71.99,72.0],[100.0,99.9,99.8002,99.7004,99.6008,99.5012,99.4018,99.3024,99.2032,99.104  …  13.0667,13.0537,13.0406,13.0276,13.0146,13.0016,12.9886,12.9756,12.9626,12.9497])

In [41]:
@time sim =sim_ind(test_onecmpt, reg, sample_times)
@time sim =sim_ind(test_onecmpt, reg, sample_times)

([0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09  …  71.91,71.92,71.93,71.94,71.95,71.96,71.97,71.98,71.99,72.0],[100.0,99.9001,99.8003,99.7006,99.601,99.5015,99.4021,99.3028,99.2036,99.1045  …  13.0623,13.0493,13.0362,13.0232,13.0102,12.9972,12.9842,12.9712,12.9582,12.9453])

 65.262 milliseconds (25304 allocations: 104 MB, 3.83% gc time)
  63.105 milliseconds (25300 allocations: 104 MB, 4.56% gc time)


In [40]:
@time sim_ind_analytical(1, 10, reg, sample_times)
@time sim_ind_analytical(1, 10, reg, sample_times)

([0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09  …  71.91,71.92,71.93,71.94,71.95,71.96,71.97,71.98,71.99,72.0],[100.0,99.9,99.8002,99.7004,99.6008,99.5012,99.4018,99.3024,99.2032,99.104  …  13.0667,13.0537,13.0406,13.0276,13.0146,13.0016,12.9886,12.9756,12.9626,12.9497])

510.643 microseconds (55 allocations: 740 KB)
 366.639 microseconds (55 allocations: 740 KB)


In [61]:
df = DataFrame(TIME = sim[1], CONC = sim[2])

function all_sims()
	reg = regimen(100., 5., interval = 12)
	p = [0.1] # KEL
	test_onecmpt(t, a, adot) = onecmptiv(t, a, adot, p)
	sample_times = [0.:0.16:reg[1][end]+24.]
	for i in 1:100
		sim =sim_ind(test_onecmpt, reg, sample_times)
		df = DataFrame(TIME = sim[1], CONC = sim[2])
		df[:i] = i
	end
    return(true)
end


all_sims (generic function with 1 method)

([0.0,12.0,24.0,36.0,48.0],[100.0,100.0,100.0,100.0,100.0])


 in depwarn at deprecated.jl:62
 in oldstyle_vcat_warning at abstractarray.jl:29
 in regimen at C:\Users\devin\Documents\Repos\julia-ode\regimen.jl:5
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at C:\Users\devin\.julia\v0.4\IJulia\src\execute_request.jl:157
 in eventloop at C:\Users\devin\.julia\v0.4\IJulia\src\IJulia.jl:123
 in anonymous at task.jl:365
while loading In[52], in expression starting on line 1


LoadError: LoadError: DimensionMismatch("index 1 has size (6,), but size(A) = ()")
while loading In[56], in expression starting on line 6


 in depwarn at deprecated.jl:62
 in oldstyle_vcat_warning at abstractarray.jl:29
 in regimen at C:\Users\devin\Documents\Repos\julia-ode\regimen.jl:5
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at C:\Users\devin\.julia\v0.4\IJulia\src\execute_request.jl:157
 in eventloop at C:\Users\devin\.julia\v0.4\IJulia\src\IJulia.jl:123
 in anonymous at task.jl:365
while loading In[56], in expression starting on line 1


In [62]:

@time all_sims()
# need y is coming out as a 2dim array so this is hack to get it to 1dim so can be a df

# in order to take advantage of parallelization need to launch julia via
# julia -p 4 (or however many processors on that comp)
#M = [regimen(100., 5, interval = 12)for i=1:100]
#@time pmap(sim_ind, M)



 in depwarn at deprecated.jl:62
 in oldstyle_vcat_warning at abstractarray.jl:29
 in regimen at C:\Users\devin\Documents\Repos\julia-ode\regimen.jl:5
 in all_sims at In[61]:4
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at C:\Users\devin\.julia\v0.4\IJulia\src\execute_request.jl:157
 in eventloop at C:\Users\devin\.julia\v0.4\IJulia\src\IJulia.jl:123
 in anonymous at task.jl:365
while loading In[62], in expression starting on line 172


true

  7.574 seconds      (1295 k allocations: 10000 MB, 2.56% gc time)
